In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#MODEL = "gpt-3.5"
MODEL = "llama2"

In [12]:
#from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama

#model = ChatOpenAI(api_key=OPENAI_API_KEY, model=MODEL)
model  = Ollama(model=MODEL)

/Users/subhanjanbasu/Desktop/local-model/rag/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [13]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/subhanjanbasu/Desktop/local-model/nutritionaladvice.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Nutritional\nAdvice\nfor\nYoung\nAdolescents\n(Boys\nand\nGirls)\nAdolescence,\nspanning\nthe\nages\nof\n10\nto\n19,\nis\na\ncritical\nperiod\nmarked\nby\nrapid\ngrowth\nand\nnumerous\nphysical,\nemotional,\nand\ncognitive\nchanges.\nProper\nnutrition\nduring\nthis\ntime\nis\nparamount\nto\nsupport\nthis\ndevelopment\nand\nestablish\na\nfoundation\nfor\nlong-term\nhealth.\nThe\ndietary\nneeds\nof\nadolescents\nare\nextensive,\nrequiring\ncareful\nattention\nto\nensure\nthey\nreceive\nadequate\nenergy ,\nmacronutrients,\nand\nmicronutrients\nto\nsupport\ntheir\ngrowth.\nEnergy\nRequirements\nEnergy\nneeds\nduring\nadolescence\nare\nsignificantly\nhigher\ncompared\nto\nother\nlife\nstages\ndue\nto\nrapid\ngrowth\nspurts\nand\nincreased\nphysical\nactivity .\nThe\naverage\ndaily\nenergy\nrequirements\nfor\nadolescent\nboys\naged\n10-18\nrange\nfrom\n2,200\nto\n3,200\nkilocalories\n(kcal)\ndepending\non\ntheir\nactivity\nlevel,\nwhile\nfor\nadolescent\ngirls\naged\n

In [14]:
def clean_text(documents):
    cleaned_text = []
    for page in documents:
        # Replace multiple newlines with a single space
        text = page.page_content.replace('\n', ' ')
        # Replace multiple spaces with a single space
        text = ' '.join(text.split())
        cleaned_text.append(text)
    return cleaned_text

# Clean the text from the pages
cleaned_pages = clean_text(pages)

# Print the cleaned text for each page
for i, page in enumerate(cleaned_pages):
    print(f"Page {i+1}:\n{page}\n")


Page 1:
Nutritional Advice for Young Adolescents (Boys and Girls) Adolescence, spanning the ages of 10 to 19, is a critical period marked by rapid growth and numerous physical, emotional, and cognitive changes. Proper nutrition during this time is paramount to support this development and establish a foundation for long-term health. The dietary needs of adolescents are extensive, requiring careful attention to ensure they receive adequate energy , macronutrients, and micronutrients to support their growth. Energy Requirements Energy needs during adolescence are significantly higher compared to other life stages due to rapid growth spurts and increased physical activity . The average daily energy requirements for adolescent boys aged 10-18 range from 2,200 to 3,200 kilocalories (kcal) depending on their activity level, while for adolescent girls aged 10-18, it ranges from 2,000 to 2,400 kcal. These requirements can vary significantly based on factors such as body size, growth rate, and 

In [18]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [15]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = DocArrayInMemorySearch.from_documents(cleaned_pages, embedding=embeddings)

In [16]:
retriever = vectorstore.as_retriever()

In [19]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
)

In [20]:
questions = [
    "What is the daily energy requirements for adolescent boys?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the daily energy requirements for adolescent boys?
Answer: According to the World Health Organization (WHO), the estimated daily energy requirements for adolescent boys are as follows:

* For sedentary males (little or no physical activity): 1700-2000 kilocalories (kcal) per day
* For moderately active males (light physical activity): 2000-2400 kcal/day
* For highly active males (heavy physical activity): 2400-3000 kcal/day

It's important to note that these are general estimates and can vary depending on factors such as body size, growth rate, and overall health status. Additionally, the dietary needs of adolescent boys may differ from those of girls due to differences in their physiology and metabolism.

It's also worth noting that these estimates are based on the energy requirements for physical growth and development, and do not take into account any additional energy needs for things like sports and other activities. As such, it's important for adolescent boys to